In [2]:
#import libraries
import os
import re
import bs4
import csv
import time
import requests
import json, codecs
from pprint import pprint
from github import Github
from bs4 import BeautifulSoup

In [3]:
def write_json(page_objects):
    with open('data.json','a') as f:
        json.dump(page_objects,f)

In [4]:
#define functions 
def find_tags(tags,each_issue):
    
    for counter,temp in enumerate(each.find_all('a')):
        tag = str(temp.string)
        tags.append(tag)

In [17]:
def find_desc(each,base_url,description,stack_trace):
    
    next_url     = base_url+each.find('a')['href']
    response     = requests.get(next_url)
    html_content = response.content
    dom          = BeautifulSoup(html_content,'html.parser')
    temp         = dom.find('div',class_ = 'edit-comment-hide')
    temp_2       = temp.find('td')
    
    #find description and the stack trace
    calculate_desc(temp_2,description,stack_trace)
    
    #find the person who closed the issue
    temp_3       = dom.find_all('div',class_='TimelineItem-body') 
    for i in temp_3:
        if i.text.find('closed this')!=-1:
            if i.find('a',class_='author Link--primary text-bold') is None:
                continue
            who_closed_it = i.find('a',class_='author Link--primary text-bold').text
    
    return who_closed_it

In [6]:
def calculate_desc(html_text,description,stack_trace):
    
    html_content = html_text.contents
    length = len(html_content)
    
    if length == 1:
        #some lines are empty. No need to write them
        if html_content[0]!= []:
            str_temp = str(html_content[0])
            if str_temp.find('java.')!=-1 or str_temp.find('AndroidRuntime:')!=-1:
                stack_trace.append(str(html_content[0]))
            else:
                description.append(str(html_content[0]))
    else:
        for i in range(length):

            if type(html_content[i]) is bs4.element.NavigableString:
                #avoid writting empty lines and html tags <br>
                if len(html_content[i])>4:
                    description.append(html_content[i])
            elif type(html_content[i] is bs4.element.Tag):
                #call recursively the function till length is 1
                calculate_desc(html_content[i],description,stack_trace)
                

In [16]:
#search all closed issues

#define some variables
total_issues  = 0
total_traces = 0

#initialize important url's
base_url = "https://github.com/"
#query_url = f"https://github.com/cgeo/cgeo/issues?page=1&q=is%3Aissue+is%3Aclosed"
query_url = f"https://github.com/cgeo/cgeo/issues?page=6&q=is%3Aissue+is%3Aclosed"

#authorization in order to make more requests.
token = os.getenv("GITHUB_TOKEN")
headers = {'Authorization': f'token {token}'}

response = requests.get(query_url, headers = headers)

In [18]:
#loop through all pages

while True:
    response_code = response.status_code
    if response_code != 200:
        raise Exception("Error Occured")
    else:
        html_content = response.content
        dom = BeautifulSoup(html_content,'html.parser')
        
        #find all issues in every page
        all = dom.findAll('div', class_='flex-auto min-width-0 p-2 pr-3 pr-md-2')
        
        page_objects = []
        
        #real scraping begins
        for each in all:
            
            #find tags and who open the issue
            tags=[]
            find_tags(tags,each)
            
            
            #find description, stack trace and who closed
            description   = []
            stack_trace   = []
            who_closed_it = find_desc(each,base_url,description,stack_trace)
            
            total_issues = total_issues+1
            if stack_trace !=[]:
                total_traces = total_traces +1
            
            #write dictionary
            if len(tags)>=1:    
                issue_object = {'name':tags[0],'tags':tags[1:len(tags)-1],'opened_by':tags[len(tags)-1],
                               'description':description,'stack_trace':stack_trace,'closed_by':who_closed_it}
           
            page_objects.append(issue_object)
            
        #write issues to json file
        write_json(page_objects)
        
        #visit the next page if exists
        end = dom.findAll('a',class_='next_page')
        
        if end ==[]:
            break
        next_url = base_url+end[1]['href']
        response = requests.get(next_url)
        # Sleep for 60 seconds
        time.sleep(60)

In [ ]:
# more generic how to find the stack trace
# export in json
# for every issue take also and the description
# take who closed the issue.===>This person possibly solve it to
########################################################################
# find total number of stack traces, find the total files
# ftiaksw parametrika diavazontas tis teleies.
# find for each file in how many stack traces appears.
#-----------------------------------------------------------------------
# predict tags in open issues
# from stack traces take the file which the bug occured
# make knowledge graph.?!later.